In [31]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [32]:
import faiss 
import numpy as np
import pickle

In [33]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [34]:
class semanticEmbedding:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    def get_embedding(self, sentences):
    # Tokenize sentences
        encoded_input = self.tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        # Perform pooling
        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings.detach().numpy()

In [35]:
class FaissIdx:

    def __init__(self, model, dim=768):
        self.index = faiss.IndexFlatIP(dim) # need to load the pickle model in the final file
        # self.index = faiss
        # Maintaining the document data
        self.doc_map = dict()
        self.model = model
        self.ctr = 0

    def add_doc(self, document_text):
        self.index.add(self.model.get_embedding(document_text))
        self.doc_map[self.ctr] = document_text # store the original document text
        self.ctr += 1

    def search_doc(self, query, k=3):
        D, I = self.index.search(self.model.get_embedding(query), 5)
        return [{self.doc_map[idx]: score} for idx, score in zip(I[0], D[0]) if idx in self.doc_map]
    def save_index(self, index_filename, doc_map_filename):
        # Save Faiss index to file
        faiss.write_index(self.index, index_filename)

        # Save document map to file using pickle
        with open(doc_map_filename, 'wb') as f:
            pickle.dump(self.doc_map, f)

    def load_index(self, index_filename, doc_map_filename):
        # Load Faiss index from file
        self.index = faiss.read_index(index_filename)

        # Load document map from file using pickle
        with open(doc_map_filename, 'rb') as f:
            self.doc_map = pickle.load(f)

In [36]:
# creating an instance of the class
model = semanticEmbedding()

In [37]:
index = FaissIdx(model)

In [38]:
index.load_index('index.bin', 'doc_map.pkl')

In [39]:
import google.generativeai as genai

In [40]:
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

In [41]:
model = genai.GenerativeModel('gemini-pro')

# given the query, run the following code sequentially

In [50]:
query = "I want to improve the situtaion of poor womens"

In [51]:
user_name = 'Amar'

In [52]:
query_result = index.search_doc(query)

In [53]:
context = ''

In [54]:
for i in range(5):
    for key, value in query_result[i].items():
        if(value>0.23):
            context += key

In [55]:
context

'Women Empowerment Scheme, Ministry of Women & Child Development = Get Information on Women Empowerment Scheme provided by the Ministry of Women & Child Development. Ministry of Women and Child Development, Government of India have come up with various schemes, programmes, social welfare schemes, Health and Nutrition, scholarship for women empowerment, Girl Child pregnant women, mothers, ward members, Anganwadi Workers, Women Health Volunteers, the women living in the rural & tribal areas, ex-servicemen, physically handicapped, nursing women, Lactating mother, widows/destitute, Old age women, women self-help group (SHG), Women Entrepreneurs and...Information on Mahila Samakhya Programme by Department of School Education and Literacy = Mahila Samakhya Programme was launched for the education and empowerment of women in rural areas. Users can get information about Mahila Samakhya Programme like funding pattern, genesis, expansion & coverage and plan document. Details of women mobilisatio

In [56]:
content = ""

In [57]:
if context == '':
    print('Is there anything else I can help you with')
else:
   content = f'''Use this query and context and respond In this format Hello [{user_name}],

      Thank you for reaching out! 🌟 Based on the information you provided, here are some government schemes that might be relevant to your situation:

      1. [Scheme Name 1]:
         - Information like eligibility, benifit, how to apply link.

      2. [Scheme Name 2]:
         - Information like eligibility, benifit, how to apply link.

      3. [Scheme Name 2]:
         - Information like eligibility, benifit, how to apply link.

      4. [Scheme Name 2]:
         - Information like eligibility, benifit, how to apply link.

      5. [Scheme Name 2]:
         - Information like eligibility, benifit, how to apply link.


      If you have any more questions or need further assistance, feel free to ask!

      Best regards,
      GovSevak.

      *This message is generated by GENAI, GENAI can make mistakes. Consider checking important information.*



      QUERY:{query} CONTEXT:{context}'''
   response = model.generate_content(content)
   print(response.text)


Hello Amar,

Thank you for reaching out! 🌟 Based on the information you provided, here are some government schemes that might be relevant to your situation:

1. **National Mission for Empowerment of Women (NMEW)**:
   - Eligibility: Women who are economically disadvantaged or marginalized.
   - Benefit: Provides financial assistance, skill development training, and other support services to help women improve their livelihoods and gain economic empowerment.
   - How to apply: Contact the nearest District Women and Child Development Office or visit the official website of the Ministry of Women and Child Development.

2. **Mahila Shakti Kendra (MSK)**:
   - Eligibility: Women of all ages and backgrounds.
   - Benefit: Provides a range of services such as skill training, counseling, legal aid, and access to government schemes to empower women and improve their quality of life.
   - How to apply: Visit the nearest Mahila Shakti Kendra or contact the local Women and Child Development Depart